In [14]:
%load_ext autoreload
%autoreload 2
# chdir to parent directory of current notebook
import sys
sys.path.append('../')

import pandas as pd
from khronos import transform

In [15]:
import os
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))


In [16]:
dataset = pd.read_csv(os.path.join(BASE_DIR, './artifacts/data/raw.csv'), index_col=0, header=[0, 1]).sort_index(axis=1)

In [18]:
flat_df = transform.flatten_dataset(dataset)
df_with_index_as_datetime = transform.convert_index_to_datetime(flat_df)
df_with_timestamp = transform.add_timestamp_in_seconds_as_raw_input(df_with_index_as_datetime)
df_with_timestamp.head()

,close,high,low,open,volume,ticker,timestamp_in_seconds
timestamp,,,,,,,
2017-09-11 09:30:00,44.01,44.05,44.01,44.01,39049.0,AAL,1505122200
2017-09-11 09:30:00,160.57,160.62,160.50,160.50,407091.0,AAPL,1505122200
2017-09-11 09:30:00,84.53,84.53,84.46,84.46,200.0,ABBV,1505122200
2017-09-11 09:30:00,83.24,83.33,83.24,83.33,13302.0,ABC,1505122200
2017-09-11 09:30:00,134.00,134.62,133.61,134.62,23371.0,ACN,1505122200


In [19]:
# store artifact as input.csv
df_with_timestamp.to_csv(os.path.join(BASE_DIR, './artifacts/data/input.csv'))

In [20]:
df_with_timestamp_loaded = pd.read_csv(os.path.join(BASE_DIR, './artifacts/data/input.csv'), index_col=0, parse_dates=True)

In [21]:
df_with_timestamp.shape

(20380661, 7)